# Parsing Indels & DSB

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.patches import Patch
import matplotlib.image as mpimg
from matplotlib import rcParams

import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import sigProfilerPlotting as sigPlt
import umap.umap_ as umap

%matplotlib inline

In [1]:
def load_ref():
    dna = {}
    for i in range(24):
        if i == 22:
            f = open('data/dna/Homo_sapiens.GRCh37.dna.chromosome.X.fa','r')
        elif i == 23:
            f = open('data/dna/Homo_sapiens.GRCh37.dna.chromosome.Y.fa','r')
        else:
            f = open('data/dna/Homo_sapiens.GRCh37.dna.chromosome.{}.fa'.format(i+1), 'r')
        content = f.read()
        seq = ''.join(content.split()[4:])
        dna[i+1] = seq
    return dna

# raw_indel = pd.read_csv('data/zou2021/denovo_indels_43genes.txt', sep='\t')
# raw_indel

dna = load_ref()

In [100]:
def reps(dna, ref, alt, length, chrom, pos):
    # Returns number of repeats of indel in Ref genome
    # CHECK
    if dna[chrom][pos] != ref[0]:
        print('NOT ALIGNED!!!')
    rep_size = 0
    up = 1
    up_pos = pos+1
    down = 1
    down_pos = pos+1

    if length > 0:
        # deletion
        indel = ref[1:]  # NOT Sure....
    else:
        # insertion
        indel = alt[1:]
        length = -length

    while up == 1 or down == 1:
        if up == 1:
            if dna[chrom][up_pos: up_pos + length] != indel:
                up = 0
            up_pos += length

        if down == 1:
            if dna[chrom][down_pos-length: down_pos] != indel:
                down = 0
            else:
                print('Downstream Rep !!!')
                print(dna[chrom][pos-length-5 : pos+length+5])
                print(ref)
                print(alt)
                print(index)
            down_pos -= length

        rep_size += up + down
        if rep_size >= 6:
            break
#     print(rep_size)
    return rep_size

def mh_len(ref, alt, length, chrom, pos):
    # Return MH-lenght (max = deletion length)
    up_length = 0
    while up_length < length:
        if dna[chrom][pos+1 + up_length] == dna[chrom][pos+1 + length + up_length]:
            up_length += 1
        else:
            break

    down_length = 0
    while down_length < length:
        if dna[chrom][pos + length - down_length] == dna[chrom][pos - down_length]:
            down_length += 1
        else:
            break
#     print(dna[chrom][pos: pos + length+10])
    return max(up_length, down_length)


In [101]:
def parse_indels(zou, dna):
    raw_indel = pd.read_csv('data/zou2021/denovo_indels_43genes.txt', sep='\t')
    # raw_indel
#     dna = load_ref()

    for index, row in raw_indel.iterrows():
        ref = row['Ref']
        alt = row['Alt']
        pos = int(row['Pos'])-1
        chrom =row['Chrom']
        if chrom == 'X':
            chrom = 23
        elif chrom =='Y':
            chrom = 24
        else:
            chrom = int(chrom)

        length = len(ref) - len(alt)
        deletion = False

        if len(ref) != 1 and len(alt) != 1:
            # COMPLEX indels?
    #         print(index+2)
    #         print(ref + ' > ' + alt)
            continue

        if length > 0:
            # DELETION
            deletion = True
            if length == 1:
                # 1bp Deletion
                rep_size = reps(dna, ref, alt, length, chrom, pos)
                idx = 1+ ( min(rep_size, 6)-1 ) * 2
                if ref[1] in ('C', 'G'):
                    idx += 1
            else:
                mh_length = mh_len(ref, alt, length, chrom, pos)

                if mh_length == 0:
                    # >1bp Deleltion with repeat size 0
                    idx = 25 + (min(length,5) - 2) * 6

                elif mh_length == length:
                    # >1bp Deletion with repeat size >= 1
                    rep_size = reps(dna, ref, alt, length, chrom, pos)
                    idx = 24 + (min(length, 5) - 2)*6 + (min(rep_size, 5))
                else:
                    # Microhomology
                    idx = 73
                    if length == 2:
                        idx = 72 + mh_length
                        # CHECK if 73
                    elif length == 3:
                        idx = 73 + mh_length
                    elif length == 4:
                        idx = 75 + mh_length
                    else:
                        idx = 78 + min(mh_length, 5)

        else:
            # INSERTION
            if length == -1:
                # 1bp Insertion
                rep_size = reps(dna, ref, alt, length, chrom, pos)
                idx = 13 + min(rep_size, 5) * 2
                if alt[1] in ('C', 'G'):
                    idx += 1

            else:
                # >1bp Insertion
                rep_size = reps(dna, ref, alt, length, chrom, pos)
                # Often rep_size 1 --> other ref genome?
                idx = 49 + (min(-length, 5) - 2) * 6 + min(rep_size, 5)

        sample = row['Ko_gene'] + '_' + row['Sample'].split('.')[1]
        if idx == 0:
            print(index)
            print(rep_size)
            print(ref)
            print(alt)
        zou.loc[sample, str(idx)] += 1


    zou['indel_count'] = zou[indel_features].sum(axis=1)
    
    return zou


    
    

# PARSE DBS

In [102]:
raw_DBS = pd.read_csv('data/zou2021/denovo_doublesub_43genes.txt', sep='\t')
# pd.set_option('display.max_rows', raw_DBS.shape[0]+1)
pd.reset_option('display.max_rows')
raw_DBS

,Sample,Ko_gene,Pathway_KO,Doublings,VariantID,Chrom,Pos,Ref,Alt,Gene,...,dist_nextSNV,Ref_nextSNV,Alt_nextSNV,dinuc_Ref,dinuc_Alt,dinuc_mutation,dinuc_Ref_rc,dinuc_Alt_rc,dinuc_mutation_rc,dinuc_mutation_final
0,MSK0.10_s2,FANCM,d_DSB repair,36,ab0b4c8c-ebf1-11e6-94f9-63d76b36f674,6,102130449,A,T,GRIK2,...,-1,C,T,AC,TT,AC>TT,GT,AA,GT>AA,AC>TT
1,MSK0.104_s1,POLI,e_TLS,12,f01aa6fc-04eb-11e7-99e7-d2e1ec9cb773,14,51298033,G,A,NIN,...,-1,A,T,GA,AT,GA>AT,TC,AT,TC>AT,GA>AT
2,MSK0.104_s1,POLI,e_TLS,12,f05c9ed6-04eb-11e7-99e7-d2e1ec9cb773,4,89411111,G,T,HERC5,...,-1,A,G,GA,TG,GA>TG,TC,CA,TC>CA,GA>TG
3,MSK0.106_s1,OGG1,j_BER,12,d5d9546a-04cc-11e7-b12e-240cd790d842,10,52209740,C,A,SGMS1,...,-1,C,A,CC,AA,CC>AA,GG,TT,GG>TT,CC>AA
4,MSK0.106_s2,OGG1,j_BER,12,e87aef0c-04ea-11e7-93a9-7d3c07da35ae,19,28546574,C,A,-,...,-1,A,T,CA,AT,CA>AT,TG,AT,TG>AT,CA>AT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,MSK0.6_s3,UNG,j_BER,12,b9cf3242-ebec-11e6-a2e4-84c56b36f674,9,101706547,C,A,COL15A1,...,-1,G,A,CG,AA,CG>AA,CG,TT,CG>TT,CG>AA
169,MSK0.6_s3,UNG,j_BER,12,b9d29270-ebec-11e6-a2e4-84c56b36f674,23,34208086,T,A,-,...,-1,G,T,TG,AT,TG>AT,CA,AT,CA>AT,CA>AT
170,MSK0.6_s3,UNG,j_BER,12,b9b6de68-ebec-11e6-a2e4-84c56b36f674,5,66999811,G,C,RP11-434D9.1,...,-1,C,T,GC,CT,GC>CT,GC,AG,GC>AG,GC>AG
171,MSK0.6_s4,UNG,j_BER,24,654c12f6-d7cb-11e6-abfc-3677e7d5a0f4,9,69514513,G,T,-,...,-1,A,T,GA,TT,GA>TT,TC,AA,TC>AA,GA>TT


In [95]:
def rev_comp(dbs):
    # Return Reverse complement of Dinucleotide substitution
    # input: NN>NN (DBS) eg AC>CG
    # return NN>NN (DBS) eg GT>CG
    rc = []
    for i in dbs.split('>'):
        dn = ''
        for n in range(len(i), 0, -1):
            if i[n-1] is 'A':
                dn += 'T'
            elif i[n-1] is 'T':
                dn += 'A'
            elif i[n-1] is 'G':
                dn += 'C'
            elif i[n-1] is 'C':
                dn += 'G'
        rc.append(dn)
    rc = '>'.join(rc)
    return rc

def parse_DBS(zou):
    xls = pd.ExcelFile('data/zou2021/DBS_features.xlsx')
    DBS_feat = pd.read_excel(xls, 'Blad1', index_col= 0,header= None)
    DBS_features = DBS_feat.index.tolist()

    raw_DBS = pd.read_csv('data/zou2021/denovo_doublesub_43genes.txt', sep='\t')
    dna = load_ref()
    for index, row in raw_DBS.iterrows():
        DBS = row['dinuc_mutation']
        if DBS not in DBS_features:
            DBS = rev_comp(DBS)
        sample = row['Ko_gene'] + '_' + row['Sample'].split('.')[1]
        zou.loc[sample, DBS] += 1   

    zou['DBS_count'] = zou[DBS_features].sum(axis=1)
#     print(zou['DBS_count'].sum(axis=0))

    return zou

In [115]:
def SBS_feat():
    zou = pd.read_pickle("data/zou2021/zou_SBS_indel_DBS.pkl")
    features = zou.columns[0:96]
    return features

def indel_feat():
    indel_features = []
    for i in range(1, 84):
        indel_features.append(str(i))
    return indel_features

def DBS_feat():
    xls = pd.ExcelFile('data/zou2021/DBS_features.xlsx')
    DBS_features = pd.read_excel(xls, 'Blad1', index_col= 0,header= None)
    DBS_features = DBS_features.index.tolist()
    return DBS_features

def load_zou():
    '''
    Generate dataframe:
    173 samples 
    96 SBSs ; gene_KO ; Count ; Protein_KO ; Subpathway_KO
    '''
    xls = pd.ExcelFile('data/zou2021/Supplementary_tables.xlsx')
    # S3 = Mutational profiles per Sample  (96 x 173)
    profiles = pd.read_excel(xls, 'TableS3', index_col= 0,header= 0)
    # S1 = Gene - Pathway mapping   (43 KOs
    KO_pathway = pd.read_excel(xls, 'TableS1', index_col= 0, header= 0)
    # zou2021.rename(columns = {"Unnamed: 0":"Sample"}, inplace=True)

    # To look-up tables Gene <-> Pathway
    pathways = {}
    genes = {}
    for index, row in KO_pathway.iterrows():
        p = row['Subpathway_KO'].split('/')
        for i in p:
            if i in pathways:
                pathways[i].append(index)
            else:
                pathways[i] = [index]
            if index in genes:
                genes[index].append(i)
            else:
                genes[index] = [i]

    features = list(profiles.index)

    profiles = profiles.drop(columns=['Mutation'])
    samples = list(profiles.columns)
    count = profiles.sum(axis=0)
    profiles = profiles.loc[:,:].div(profiles.sum(axis=0), axis=1)
    profiles = profiles.transpose()
    gene_KO = []
    for s in samples:
        gene_KO.append(s.split('_')[0])
    profiles['Gene_KO'] = gene_KO
    profiles['Count'] = count
    joined = pd.merge(left=profiles, right=KO_pathway, how= 'left', left_on='Gene_KO', right_index = True)

    x = profiles.loc[:,features].values  # Separating out the target
    y = profiles.loc[:, 'Gene_KO'].values  # Standardizing the features

#     print(joined.shape)
    
    indel_features = []
    for i in range(1, 84):
        indel_features.append(str(i))
    joined = joined.assign(**dict.fromkeys(indel_features, 0))
    
    xls = pd.ExcelFile('data/zou2021/DBS_features.xlsx')
    DBS_features = pd.read_excel(xls, 'Blad1', index_col= 0,header= None)
    DBS_features = DBS_features.index.tolist()
    joined = joined.assign(**dict.fromkeys(DBS_features, 0))
    
    joined = parse_indels(joined, dna)
    joined = parse_DBS(joined)
    
#     profiles = profiles.loc[:,:].div(profiles.sum(axis=0), axis=1)
    
    joined[indel_feat()] = joined[indel_feat()].div(joined['indel_count'].values,axis=0)
    
    joined[DBS_feat()] = joined[DBS_feat()].div(joined['DBS_count'].values,axis=0)
    
    return joined


# Add columns for Indels
indel_features = []
for i in range(1, 84):
    indel_features.append(str(i))

    
zou = load_zou()
zou


zou.to_pickle("data/zou2021/zou_96SBS_83indel_78DBS.pkl")




Downstream Rep !!!
ATCCATGAAAAT
G
GG
172
Downstream Rep !!!
GCATCTATTTTT
A
AA
172


In [121]:
zou = pd.read_pickle("data/zou2021/zou_96SBS_83indel_78DBS.pkl")

zou = zou.drop(columns=DBS_feat())
zou = zou.dropna(axis=0)


zou.to_pickle("data/zou2021/zou_96SBS_83indel.pkl")
zou


,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,A[C>G]A,A[C>G]C,A[C>G]G,A[C>G]T,A[C>T]A,A[C>T]C,...,76,77,78,79,80,81,82,83,indel_count,DBS_count
ATM_148_s1,0.054152,0.003610,0.003610,0.014440,0.007220,0.000000,0.000000,0.003610,0.018051,0.007220,...,0.0,0.071429,0.000000,0.071429,0.071429,0.000000,0.000000,0.000000,14,1
ATM_148_s2,0.054902,0.000000,0.007843,0.039216,0.003922,0.000000,0.000000,0.000000,0.031373,0.003922,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,9,0
ATM_16_s1,0.100000,0.002778,0.005556,0.027778,0.002778,0.000000,0.000000,0.008333,0.016667,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,6,1
ATM_16_s2,0.062731,0.007380,0.000000,0.029520,0.007380,0.000000,0.003690,0.003690,0.014760,0.011070,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,19,0
ATP2B4_2_s3,0.069620,0.012658,0.000000,0.006329,0.000000,0.006329,0.000000,0.000000,0.037975,0.012658,...,0.0,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WRN_56_s4,0.046392,0.005155,0.015464,0.015464,0.010309,0.005155,0.000000,0.000000,0.020619,0.005155,...,0.0,0.000000,0.000000,0.166667,0.111111,0.055556,0.000000,0.000000,18,0
XRCC4_77_s1,0.037815,0.000000,0.004202,0.012605,0.012605,0.000000,0.004202,0.000000,0.033613,0.016807,...,0.0,0.000000,0.090909,0.000000,0.000000,0.000000,0.090909,0.000000,11,1
XRCC4_77_s2,0.092166,0.004608,0.000000,0.018433,0.000000,0.004608,0.000000,0.004608,0.023041,0.004608,...,0.0,0.000000,0.000000,0.071429,0.000000,0.000000,0.071429,0.000000,14,0
XRCC4_78_s1,0.071918,0.003425,0.000000,0.030822,0.003425,0.006849,0.003425,0.000000,0.020548,0.013699,...,0.0,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,17,0


# TEMP

In [128]:
zou = pd.read_pickle("data/zou2021/zou_96SBS_83indel_78DBS.pkl")

zou = zou.drop(columns=DBS_feat())
zou = zou.dropna(axis=0)

## Insertions
ins = []
for i in range(13,25,1):
    ins.append(str(i))
for i in range(49,73,1):
    ins.append(str(i))
print(ins)

zou['ins/indel'] = zou[ins].sum(axis=1)
zou['indel/SBS'] = zou['indel_count'].div(zou['Count'].values,axis=0)

zou.to_pickle("data/zou2021/zou_96SBS_83indel.pkl")
zou

['13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72']


,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,A[C>G]A,A[C>G]C,A[C>G]G,A[C>G]T,A[C>T]A,A[C>T]C,...,78,79,80,81,82,83,indel_count,DBS_count,ins/indel,indel/SBS
ATM_148_s1,0.054152,0.003610,0.003610,0.014440,0.007220,0.000000,0.000000,0.003610,0.018051,0.007220,...,0.000000,0.071429,0.071429,0.000000,0.000000,0.000000,14,1,0.500000,0.050542
ATM_148_s2,0.054902,0.000000,0.007843,0.039216,0.003922,0.000000,0.000000,0.000000,0.031373,0.003922,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,9,0,0.333333,0.035294
ATM_16_s1,0.100000,0.002778,0.005556,0.027778,0.002778,0.000000,0.000000,0.008333,0.016667,0.000000,...,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,6,1,0.500000,0.016667
ATM_16_s2,0.062731,0.007380,0.000000,0.029520,0.007380,0.000000,0.003690,0.003690,0.014760,0.011070,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,19,0,0.631579,0.070111
ATP2B4_2_s3,0.069620,0.012658,0.000000,0.006329,0.000000,0.006329,0.000000,0.000000,0.037975,0.012658,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,6,1,0.166667,0.037975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WRN_56_s4,0.046392,0.005155,0.015464,0.015464,0.010309,0.005155,0.000000,0.000000,0.020619,0.005155,...,0.000000,0.166667,0.111111,0.055556,0.000000,0.000000,18,0,0.277778,0.092784
XRCC4_77_s1,0.037815,0.000000,0.004202,0.012605,0.012605,0.000000,0.004202,0.000000,0.033613,0.016807,...,0.090909,0.000000,0.000000,0.000000,0.090909,0.000000,11,1,0.454545,0.046218
XRCC4_77_s2,0.092166,0.004608,0.000000,0.018433,0.000000,0.004608,0.000000,0.004608,0.023041,0.004608,...,0.000000,0.071429,0.000000,0.000000,0.071429,0.000000,14,0,0.357143,0.064516
XRCC4_78_s1,0.071918,0.003425,0.000000,0.030822,0.003425,0.006849,0.003425,0.000000,0.020548,0.013699,...,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,17,0,0.352941,0.058219
